# Download Sentinel-2 Geomedians


## Description

Our partners in Africa require S2 basemaps to assist with gathering more training data for the crop-mask. This notebook will extract S2 semi-annual geomedians for an AEZ. The results should be downloaded an imported into a GIS platform for viewing


***

If dask worker memory limits are an issue, try adding this to the dask.distribute yaml:

put below into `~/.config/dask/distributed.yaml`

    distributed:
      worker:
        memory:
          target: False  # Avoid spilling to disk
          spill: False  # Avoid spilling to disk
          pause: 0.93  # fraction at which we pause worker threads
          terminate: 0.95  # fraction at which we terminate the worker


Can check this has worked by running the following in a cell:

`dask.config.config['distributed']['worker']['memory']`
***

### Load packages


In [ ]:
# !pip install git+https://github.com/digitalearthafrica/deafrica-sandbox-notebooks.git@minty-fresh-sandbox#subdirectory=Tools

In [ ]:
%matplotlib inline

import xarray as xr
import geopandas as gpd
import numpy as np
import datacube
import dask
from datacube.utils.cog import write_cog
import matplotlib.pyplot as plt
from datacube.utils import geometry

from deafrica_tools.plotting import rgb, display_map, map_shapefile
from deafrica_tools.dask import create_local_dask_cluster

In [ ]:
create_local_dask_cluster()

### Connect to the datacube

In [ ]:
dc = datacube.Datacube(app='s2_gm')

## Analysis Parameters

In [ ]:
product='gm_s2_semiannual'
aez_region = '../testing/southern_cropmask/data/Southern.shp'
chunks={'x':2000,'y':2000}
time_range=('2019')

## View the Area of Interest on an Interactive map

In [ ]:
aez = gpd.read_file(aez_region)

map_shapefile(aez, attribute='COUNTRY')

## Load Sentinel-2 semiannual GMs

In [ ]:
# Create a reusable query
query = {
    'time': time_range,
    'resolution': (-10, 10),
    'measurements': ['red','green','blue'],
    'output_crs': 'epsg:6933',
    'dask_chunks':chunks,
    'resampling':'average'
}

# set up query based on polygon (convert to WGS84)
geom = geometry.Geometry(aez.geometry.values[0].__geo_interface__,
                         geometry.CRS('epsg:4326'))

# print(geom)
q = {"geopolygon": geom}

# merge polygon query with user supplied query params
query.update(q)

ds = dc.load(product=product, **query)
# ds = ds.compute()
print(ds)

## Write RGB COG to disk

In [ ]:
write_cog(ds.isel(time=0).to_array().compute(),
          fname='s2_gm_jan_jun_2019.tif',
          overwrite=True)

In [ ]:
write_cog(ds.isel(time=1).to_array().compute(),
          fname='s2_gm_jul_dec_2019.tif',
          overwrite=True)